In [1]:
library(FigR)
library(Seurat)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(foreach)

Loading required package: Matrix

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats,

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/Jurkat/Jurkat/Kendall/Pairs.tsv.gz"
path.pairs.ABC = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/Jurkat/Jurkat/multiome_powerlaw_v3/encode_e2g_predictions.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/Jurkat/Jurkat/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/Jurkat/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/Jurkat/1.Genome_wide_prediction/FigR/FigR.250321/"
celltype = "Jurkat"

In [3]:
n.cores = 8

Import candidate E-G pairs

In [4]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ABC results

In [5]:
pairs.ABC = readGeneric(path.pairs.ABC,
                        header = T,
                        keep.all.metadata = T)

Filter ABC results which distance < 1M

In [6]:
pairs.ABC.1M = pairs.ABC[pairs.ABC$distance < 10^6]

Filter E-G pairs overlaping with pairs.ABC.1M

In [7]:
df.pairs.E2G.chr_rename = as.data.frame(pairs.E2G)[,1:3]
df.pairs.E2G.chr_rename[,"seqnames"] = paste(seqnames(pairs.E2G),
                                             mcols(pairs.E2G)[,"TargetGene"],
                                             sep = "_")
pairs.E2G.chr_rename = GRanges(df.pairs.E2G.chr_rename)
rm(df.pairs.E2G.chr_rename)

df.pairs.ABC.1M.chr_rename = as.data.frame(pairs.ABC.1M)[,1:3]
df.pairs.ABC.1M.chr_rename[,"seqnames"] = paste(seqnames(pairs.ABC.1M),
                                                mcols(pairs.ABC.1M)[,"TargetGene"],
                                                sep = "_")
pairs.ABC.1M.chr_rename = GRanges(df.pairs.ABC.1M.chr_rename)
rm(df.pairs.ABC.1M.chr_rename)

pairs.E2G.filter = pairs.E2G[countOverlaps(pairs.E2G.chr_rename,
                                           pairs.ABC.1M.chr_rename) > 0]
rm(pairs.E2G.chr_rename)
rm(pairs.ABC.1M.chr_rename)
pairs.E2G.filter

GRanges object with 2793593 ranges and 3 metadata columns:
            seqnames            ranges strand |  TargetGene
               <Rle>         <IRanges>  <Rle> | <character>
        [1]     chr1 10001055-10001570      * | APITD1-CORT
        [2]     chr1 10001055-10001570      * |       CASZ1
        [3]     chr1 10001055-10001570      * |       CENPS
        [4]     chr1 10001055-10001570      * |      CLSTN1
        [5]     chr1 10001055-10001570      * |    CTNNBIP1
        ...      ...               ...    ... .         ...
  [2793589]     chrX   9941883-9942743      * |        WWC3
  [2793590]     chrX 99603024-99603776      * |      PCDH19
  [2793591]     chrX 99632013-99632219      * |      PCDH19
  [2793592]     chrX   9995829-9996207      * |       TBL1X
  [2793593]     chrX   9995829-9996207      * |        WWC3
                          PeakName               PairName
                       <character>            <character>
        [1] chr1-10001055-10001570 chr1-10001

Import ATAC matrix

In [8]:
matrix.atac_count = readRDS(path.matrix.atac_count)
matrix.atac = centerCounts(matrix.atac_count)

Matrix object input detectedCentering counts for cells sequentially in groups of size  1000  ..

Computing centered counts for cells:  1  to  1000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  1001  to  2000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  2001  to  3000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  3001  to  4000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  4001  to  5000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  5001  to  5277 ..
Computing centered counts per cell using mean reads in features ..

Merging results..
Done!


Import RNA matrix

In [9]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = matrix.rna_count[rowSums(matrix.rna_count) > 0,]
matrix.rna = NormalizeData(matrix.rna)
rm(matrix.rna_count)

Map gene names

In [10]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
    library(dplyr)
    library(data.table)
    
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [11]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:SummarizedExperiment’:

    shift


The following object is masked from ‘package:GenomicRanges’:

    shift


The following object is masked from ‘package:IRanges’:

    shift


The following objects are masked from ‘package:S4Vectors’:

    first, second




In [12]:
pairs.E2G.filter2 = pairs.E2G.filter[pairs.E2G.filter$TargetGene %in% rownames(matrix.rna.rename) &
                                     pairs.E2G.filter$PeakName %in% rownames(matrix.atac)]

Prepare FigR input data

In [13]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak$PeakName = paste(seqnames(bed.peak),
                          start(bed.peak),
                          end(bed.peak),
                          sep = "-")
names(bed.peak) = bed.peak$PeakName
bed.peak = bed.peak[rownames(matrix.atac)]

In [14]:
ATAC.se = SummarizedExperiment(assays = SimpleList(counts = matrix.atac_count),
                               rowRanges = bed.peak)
rm(matrix.atac_count)
ATAC.se <- chromVAR::addGCBias(ATAC.se, genome = BSgenome.Hsapiens.UCSC.hg38)
ATAC.se

class: RangedSummarizedExperiment 
dim: 152887 5277 
metadata(0):
assays(1): counts
rownames(152887): chr1-10001055-10001570 chr1-100037799-100039086 ...
  chrX-99632013-99632219 chrX-9995829-9996207
rowData names(2): PeakName bias
colnames(5277): e10l1_AAACCAGGTACTTATG-1 e10l1_AAACCAGGTCACACTG-1 ...
  e10l2_TTTGTGAGTCCTGAGC-1 e10l2_TTTGTGAGTTTATGTG-1
colData names(0):

In [15]:
bg <- chromVAR::getBackgroundPeaks(ATAC.se, niterations = 100)

In [16]:
index.gene = data.frame(gene_name = rownames(matrix.rna.rename),
                       index = 1:nrow(matrix.rna.rename))
rownames(index.gene) = index.gene$gene_name

index.peak = data.frame(peak_name = rownames(matrix.atac),
                       index = 1:nrow(matrix.atac))
rownames(index.peak) = index.peak$peak_name

Run FigR prediction

In [17]:
Sys.setenv(OPENBLAS_NUM_THREADS=1)
PeakGeneCor.modified = function (ATAC, RNA, OV, ncores = 4, chunkSize = 200, metric = "spearman", 
    bg = NULL) 
{
    stopifnot(ncol(ATAC) == ncol(RNA))
    if (chunkSize > 1000) 
        stop("Do not specify very large chunk sizes. Please use chunkSize < 1000")
    # n <- length(OV)
    n <- nrow(OV) # import OV as a dataframe instead of Hits object
    starts <- seq(1, n, chunkSize)
    ends <- starts + chunkSize - 1
    ends[length(ends)] <- n
    # OVd <- OV %>% as.data.frame() %>% dplyr::rename(Gene = "queryHits", 
    #     Peak = "subjectHits")
    OVd <- OV %>% dplyr::rename(Gene = "queryHits", Peak = "subjectHits")
    chunkList <- mapply(c, starts, ends, SIMPLIFY = FALSE)
    time_elapsed <- Sys.time()
    cat("Running in parallel using ", ncores, "cores ..\n")
    cat("Computing observed correlations ..\n")
    corList <- pbmcapply::pbmclapply(X = chunkList, FUN = function(x) {
        FigR::chunkCore(chunk = x, A = ATAC, R = RNA, O = OVd, 
            met = metric)
    }, mc.cores = ncores)
    if (any(unlist(sapply(corList, is.null)))) {
        message("One or more of the chunk processes failed unexpectedly (returned NULL) ..")
        message("Please check to see you have enough cores/memory allocated")
        message("Also make sure you have filtered down to non-zero peaks/genes")
    }
    OVd$rObs <- unlist(corList)
    cat("Finished!\n")
    time_elapsed <- Sys.time() - time_elapsed
    cat(paste("\nTime Elapsed: ", time_elapsed, units(time_elapsed)), 
        "\n\n")
    if (!is.null(bg)) {
        n_iter <- ncol(bg)
        cat("Computing background correlations ..\n")
        time_elapsed <- Sys.time()
        bgCor <- foreach(i = 1:n_iter, .combine = "cbind", .export = c("chunkCore", 
            "t"), .packages = c("pbmcapply", "FigR", "Matrix")) %do% 
            {
                OVdBg <- OVd[, 1:2]
                OVdBg$Peak <- bg[OVdBg$Peak, i]
                bgCorList <- pbmcapply::pbmclapply(X = chunkList, 
                  FUN = function(x) {
                    chunkCore(chunk = x, A = ATAC, R = RNA, O = OVdBg, 
                      met = metric)
                  }, mc.cores = ncores)
                unlist(bgCorList)
            }
        if (sum(is.null(bgCor)) != 0 | sum(is.na(bgCor)) != 0) 
            stop("One or more of the chunk processes failed unexpectedly (returned NULL) .. Please check to see you have enough cores/m\n           emory allocated")
        time_elapsed <- Sys.time() - time_elapsed
        cat(paste("\nTime Elapsed: ", time_elapsed, units(time_elapsed)), 
            "\n\n")
        colnames(bgCor) <- paste0("rBg", 1:ncol(bgCor))
        OVd <- cbind(OVd, bgCor)
    }
    return(OVd)
}

In [18]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [19]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G.filter2)))
chr.run = chr.run[!chr.run %in% chr.done]

In [20]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr.res = pairs.E2G.filter2[seqnames(pairs.E2G.filter2) == chr.tmp]
  genePeakOv.chr <- data.frame(queryHits = index.gene[pairs.E2G.chr.res$TargetGene, "index"],
                               subjectHits = index.peak[pairs.E2G.chr.res$PeakName, "index"])
  ObsCor.chr = PeakGeneCor.modified(ATAC = matrix.atac, 
                                    RNA = matrix.rna.rename,
                                    OV = genePeakOv.chr, 
                                    chunkSize = 500,
                                    ncores = n.cores, 
                                    bg = bg)
  pairs.E2G.chr.res$rObs = ObsCor.chr[,"rObs"]
  pairs.E2G.chr.res$rBgSD <- matrixStats::rowSds(as.matrix(ObsCor.chr[, 4:103]))
  pairs.E2G.chr.res$rBgMean <- rowMeans(ObsCor.chr[, 4:103])
  pairs.E2G.chr.res$pvalZ <- 1 - stats::pnorm(q = pairs.E2G.chr.res$rObs, 
                                              mean = pairs.E2G.chr.res$rBgMean,
                                              sd = pairs.E2G.chr.res$rBgSD)
  saveRDS(pairs.E2G.chr.res,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.res)
}

[1] "chr1"
Running in parallel using  8 cores ..
Computing observed correlations ..
Finished!

Time Elapsed:  16.1432342529297 secs 

Computing background correlations ..


Loading required package: parallel




Time Elapsed:  30.0909704764684 mins 

[1] "chr10"
Running in parallel using  8 cores ..
Computing observed correlations ..
Finished!

Time Elapsed:  10.121141910553 secs 

Computing background correlations ..

Time Elapsed:  11.9289192756017 mins 

[1] "chr11"
Running in parallel using  8 cores ..
Computing observed correlations ..
Finished!

Time Elapsed:  13.3429152965546 secs 

Computing background correlations ..

Time Elapsed:  20.7545115788778 mins 

[1] "chr12"
Running in parallel using  8 cores ..
Computing observed correlations ..
Finished!

Time Elapsed:  10.3885886669159 secs 

Computing background correlations ..

Time Elapsed:  15.1394369324048 mins 

[1] "chr13"
Running in parallel using  8 cores ..
Computing observed correlations ..
Finished!

Time Elapsed:  2.86807775497437 secs 

Computing background correlations ..

Time Elapsed:  5.57543821334839 mins 

[1] "chr14"
Running in parallel using  8 cores ..
Computing observed correlations ..
Finished!

Time Elapsed:  5.

Merge results

In [21]:
chr.merge = as.character(unique(seqnames(pairs.E2G.filter2)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [22]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 2590409 ranges and 7 metadata columns:
       seqnames            ranges strand |  TargetGene               PeakName
          <Rle>         <IRanges>  <Rle> | <character>            <character>
  chr1     chr1 10001055-10001570      * | APITD1-CORT chr1-10001055-10001570
  chr1     chr1 10001055-10001570      * |       CASZ1 chr1-10001055-10001570
  chr1     chr1 10001055-10001570      * |      CLSTN1 chr1-10001055-10001570
  chr1     chr1 10001055-10001570      * |    CTNNBIP1 chr1-10001055-10001570
  chr1     chr1 10001055-10001570      * |        DFFA chr1-10001055-10001570
   ...      ...               ...    ... .         ...                    ...
  chrX     chrX   9941883-9942743      * |        WWC3   chrX-9941883-9942743
  chrX     chrX 99603024-99603776      * |      PCDH19 chrX-99603024-99603776
  chrX     chrX 99632013-99632219      * |      PCDH19 chrX-99632013-99632219
  chrX     chrX   9995829-9996207      * |       TBL1X   chrX-9995829-9996207
  chr

In [23]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "rObs",
                         "rBgSD",
                         "rBgMean",
                         "pvalZ")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,rObs,rBgSD,rBgMean,pvalZ
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10001055,10001570,APITD1-CORT,Jurkat,-0.0071411282,0.014130634,-3.175940e-04,0.68541289
chr1,10001055,10001570,CASZ1,Jurkat,-0.0022533046,0.015386062,3.392746e-03,0.64317512
chr1,10001055,10001570,CLSTN1,Jurkat,0.0197653646,0.013797346,4.698342e-04,0.08098212
chr1,10001055,10001570,CTNNBIP1,Jurkat,-0.0277141855,0.014525003,2.252927e-03,0.98045033
chr1,10001055,10001570,DFFA,Jurkat,-0.0185396358,0.014258860,-5.241866e-04,0.89678741
chr1,10001055,10001570,GPR157,Jurkat,0.0148254118,0.013936212,1.939612e-03,0.17757994
chr1,10001055,10001570,H6PD,Jurkat,-0.0050693443,0.014255930,8.466787e-04,0.66092423
chr1,10001055,10001570,KIF1B,Jurkat,0.0107859814,0.013129322,-1.285664e-05,0.20539682
chr1,10001055,10001570,LINC01759,Jurkat,-0.0019488251,0.008548509,-1.129132e-03,0.53819492


In [24]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  grid      stats4    stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] pbmcapply_1.5.1                   data.table_1.15.2                
 [3] foreach_1.5.2                     BSgenome.H